# Fetching

## Import useful libraries

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

## Declare constants and read the API key

In [2]:
load_dotenv()
API_KEY = os.getenv("FMP_API_KEY")

BASE_URL = "https://financialmodelingprep.com/api/v3"
LIMIT = 80  # Number of periods (80 quarters = 20 years)
PERIOD = "quarter"  # Fetch quarterly data
OUTPUT_DIR = "data"
OUTPUT_FILENAME = "balance_sheets_quarterly.csv"
EXCHANGE_SHORT_NAME = "WSE"  # Warsaw Stock Exchange short name

## Useful functions for data fetching

In [3]:
def fetch_all_symbols():
    url = f"{BASE_URL}/stock/list?apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data


def filter_wse_symbols(all_symbols):
    wse_symbols = [
        stock["symbol"]
        for stock in all_symbols
        if stock["exchangeShortName"] == EXCHANGE_SHORT_NAME
    ]
    return wse_symbols


def fetch_balance_sheet(symbol):
    url = f"{BASE_URL}/balance-sheet-statement/{symbol}?apikey={API_KEY}&limit={LIMIT}&period={PERIOD}"
    response = requests.get(url)
    data = response.json()
    return data


def fetch_historical_stock_prices(symbol, from_date, to_date):
    # Fetch historical prices for the exact date range of balance sheets
    url = f"{BASE_URL}/historical-price-full/{symbol}?from={from_date}&to={to_date}&apikey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if "historical" in data:
        return data["historical"]
    return []


def get_closest_price(stock_prices, date):
    # Find the closest stock price to the balance sheet date
    for price_record in stock_prices:
        if price_record["date"] <= date:
            return price_record["close"]
    return None


def create_dataframe(symbols):
    all_data = []

    for symbol in symbols:
        balance_sheet_data = fetch_balance_sheet(symbol)

        if not balance_sheet_data:
            continue

        # Get the earliest and latest dates from the balance sheet data
        balance_sheet_dates = [record["date"] for record in balance_sheet_data]
        from_date = min(balance_sheet_dates)
        to_date = max(balance_sheet_dates)

        # Fetch historical prices for the date range
        stock_prices = fetch_historical_stock_prices(symbol, from_date, to_date)

        for record in balance_sheet_data:
            date = record["date"]
            stock_price = get_closest_price(
                stock_prices, date
            )  # Find the closest price to the balance sheet date
            shares_outstanding = record.get(
                "commonStock", None
            )  # Get the number of shares outstanding

            if stock_price is not None and shares_outstanding is not None:
                # Calculate market cap as stock price * shares outstanding
                market_cap = stock_price * shares_outstanding
            else:
                market_cap = None

            record["symbol"] = symbol
            record["marketCap"] = market_cap
            all_data.append(record)

    df = pd.DataFrame(all_data)
    return df

## Fetch data and create a DataFrame

In [4]:
all_symbols = fetch_all_symbols()
all_symbols

[{'symbol': 'PMGOLD.AX',
  'name': 'Perth Mint Gold',
  'price': 17.94,
  'exchange': 'Australian Securities Exchange',
  'exchangeShortName': 'ASX',
  'type': 'etf'},
 {'symbol': 'ANGL.TA',
  'name': 'Salomon A. Angel Ltd.',
  'price': 3382,
  'exchange': 'Tel Aviv',
  'exchangeShortName': 'TLV',
  'type': 'stock'},
 {'symbol': 'PRPD.TA',
  'name': 'Propdo Ltd',
  'price': 2037,
  'exchange': 'Tel Aviv',
  'exchangeShortName': 'TLV',
  'type': 'stock'},
 {'symbol': 'ZUR.TA',
  'name': 'Zur Shamir Holdings Ltd',
  'price': 613.2,
  'exchange': 'Tel Aviv',
  'exchangeShortName': 'TLV',
  'type': 'stock'},
 {'symbol': 'SPDV.TA',
  'name': 'Speedvalue Ltd',
  'price': 360.3,
  'exchange': 'Tel Aviv',
  'exchangeShortName': 'TLV',
  'type': 'stock'},
 {'symbol': 'TEDE.TA',
  'name': 'Tedea Technological Development and Automation Ltd.',
  'price': 632,
  'exchange': 'Tel Aviv',
  'exchangeShortName': 'TLV',
  'type': 'stock'},
 {'symbol': 'MEDN.TA',
  'name': 'Mehadrin Ltd.',
  'price': 14

In [5]:
wse_symbols = filter_wse_symbols(all_symbols)
wse_symbols

['PEO.WA',
 'NTC.WA',
 'ICE.WA',
 'CDR.WA',
 'RLP.WA',
 'JSW.WA',
 'KGH.WA',
 'TPE.WA',
 'EAT.WA',
 'ULG.WA',
 'PXM.WA',
 'NNG.WA',
 'CCC.WA',
 'SNT.WA',
 'AMB.WA',
 'DNP.WA',
 'MOC.WA',
 'GRX.WA',
 'DAD.WA',
 'PUR.WA',
 'HRS.WA',
 'PEP.WA',
 'EUR.WA',
 'MDG.WA',
 'CPS.WA',
 'PKP.WA',
 'KER.WA',
 'ETFBW20TR.WA',
 'RBW.WA',
 'ETFBCASH.WA',
 'ETFBW20ST.WA',
 'ETFSP500.WA',
 'ETFBW20LV.WA',
 'MIL.WA',
 '11B.WA',
 'LBW.WA',
 'APT.WA',
 'MRB.WA',
 'LPP.WA',
 'CIG.WA',
 'CLN.WA',
 'CBF.WA',
 'PTG.WA',
 'PGE.WA',
 'ALR.WA',
 'PCO.WA',
 'UNT.WA',
 'BDX.WA',
 'PZU.WA',
 'PKO.WA',
 'GPP.WA',
 'FAB.WA',
 'RAE.WA',
 'BRS.WA',
 'CLC.WA',
 'XTB.WA',
 'MRC.WA',
 'GTN.WA',
 'COG.WA',
 'ENA.WA',
 'ALE.WA',
 'SKL.WA',
 'DBC.WA',
 'KPL.WA',
 'GMT.WA',
 'GIF.WA',
 'KGN.WA',
 'VVD.WA',
 'CMR.WA',
 'BBD.WA',
 'SFG.WA',
 'APN.WA',
 'PLW.WA',
 'OPN.WA',
 'ABE.WA',
 'MLG.WA',
 'INC.WA',
 'PAS.WA',
 'VRG.WA',
 'SNX.WA',
 'ACG.WA',
 'ENI.WA',
 'OPL.WA',
 'GRN.WA',
 'BFT.WA',
 'BIO.WA',
 'AMC.WA',
 'OTS.WA',
 'MA

In [6]:
df = create_dataframe(wse_symbols)

In [7]:
df.head()

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,cashAndCashEquivalents,shortTermInvestments,...,totalEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,link,finalLink,marketCap
0,2024-06-30,PEO.WA,PLN,0000000000,2024-06-30,2024-06-28 20:00:00,2024,Q2,1.552300e+10,-2.850300e+10,...,2.822300e+10,3.160460e+11,12000000.0,3.160460e+11,4.920000e+08,1.981900e+10,4.296000e+09,,,4.401600e+10
1,2024-03-31,PEO.WA,PLN,0000000000,2024-03-31,2024-03-30 20:00:00,2024,Q1,1.691600e+10,0.000000e+00,...,3.168300e+10,3.173690e+11,13000000.0,3.173690e+11,1.266490e+11,2.042600e+10,3.510000e+09,,,4.764470e+10
2,2023-12-31,PEO.WA,PLN,0000000000,2023-12-31,2023-12-30 19:00:00,2023,Q4,1.473300e+10,0.000000e+00,...,3.034100e+10,3.057230e+11,12000000.0,3.057230e+11,1.198370e+11,1.858300e+10,3.850000e+09,,,3.983710e+10
3,2023-09-30,PEO.WA,PLN,0000000000,2023-09-30,2023-09-29 20:00:00,2023,Q3,1.131247e+10,0.000000e+00,...,2.833568e+10,3.099348e+11,12009000.0,3.099348e+11,1.210954e+11,1.808865e+10,6.776180e+09,,,2.649635e+10
4,2023-06-30,PEO.WA,PLN,0000000000,2023-06-30,2023-06-29 20:00:00,2023,Q2,9.912213e+09,0.000000e+00,...,2.596572e+10,2.879213e+11,11270000.0,2.879213e+11,1.033854e+11,1.606473e+10,6.152515e+09,,,2.908168e+10


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20378 entries, 0 to 20377
Data columns (total 55 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   date                                     20378 non-null  object 
 1   symbol                                   20378 non-null  object 
 2   reportedCurrency                         20378 non-null  object 
 3   cik                                      20378 non-null  object 
 4   fillingDate                              20378 non-null  object 
 5   acceptedDate                             20146 non-null  object 
 6   calendarYear                             20378 non-null  object 
 7   period                                   20378 non-null  object 
 8   cashAndCashEquivalents                   20378 non-null  float64
 9   shortTermInvestments                     20378 non-null  float64
 10  cashAndShortTermInvestments              20378

In [9]:
# Ensure the output directory exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Save DataFrame to a CSV file in the output directory
output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)
df.to_csv(output_path, index=False)

print(f"Data has been saved to {output_path}")

Data has been saved to data\balance_sheets_quarterly.csv
